In [ ]:
!pip install pandas
!pip install numpy
!pip install pydirectory
!pip install python-chess


In [ ]:
import pandas as pd
import numpy 
import chess
import chess.pgn
import chess.engine
import random


In [ ]:
# THIS SECTION CONVERTS THE VISUAL BOARD TO USABLE DATA
# PLACES BLACK AND WHITE INTO THE SAME MATRICES


# defines positions on the board
squares_index = {
  'a': 0,
  'b': 1,
  'c': 2,
  'd': 3,
  'e': 4,
  'f': 5,
  'g': 6,
  'h': 7
}


# function converts board position to a linear 
# example: h3 -> [7][3]
def square_to_index(square):
  letter = chess.square_name(square)                  # converts the letter position into a number
  return 8 - int(letter[1]), squares_index[letter[0]] # return in format of [col][row]


def split_dims(board):
    # this is the 3d matrix
    # converts board state into usable data
    # 12 boards for 8x8 position of piece types, 6 for white followed by 6 for black
    # pieces in order of pawns, knights, bishops, rooks, queen, king
    # last 2 boards are attack positions for white and then black
    
  board3d = numpy.zeros((6, 8, 8), dtype=numpy.int8) # 12 by 8 by 8 vector of all 0's

  # provides pieces's view in the matrix
  for piece in chess.PIECE_TYPES:                    # for a specific piece type (pawn, etc)
    for square in board.pieces(piece, chess.WHITE):  # for a specific tile on the board, checking for White's pieces only
      idx = numpy.unravel_index(square, (8, 8))      # convert array into a grid for viewing
      board3d[piece - 1][7 - idx[0]][idx[1]] = -1    # sets the matrix position to -1 if true, 0 if piece is not present
    for square in board.pieces(piece, chess.BLACK):  # for a specific tile on the board, checking for Black's pieces only
      idx = numpy.unravel_index(square, (8, 8))      # convert array into a grid for viewing
      board3d[piece - 1][7 - idx[0]][idx[1]] = 1     # sets the matrix position to 1 if piece is present


  return board3d

#For the above source code (now altered):
#/***************************************************************************************
#*    Title: Chess-AI-with-TensorFlow
#*    Author: Nihal Puram
#*    Date: September 11th, 2021
#*    Code version: Unknown
#*    Availability: https://github.com/realnihal/Chess-AI-with-TensorFlow
#*
#***************************************************************************************/

In [ ]:
# PIECE MOVEMENTS
# FUNCTION TO CONVERT LEGAL MOVES INTO MATRICES


def splitLegalMove(board):
    
    allPositions = split_dims(board)
    matrix = numpy.zeros((6, 8, 8), dtype=numpy.int8) # empty array to be used as output
    flatPos = numpy.zeros((8, 8), dtype=numpy.int8) # single matrix of all pieces 
    numBPieces = numpy.zeros(6, dtype=numpy.int8)
    
    for i in range (6):
        flatPos = flatPos + allPositions[i]
    flatPos = numpy.flip(flatPos,0)

    numP = 0
    numN = 0
    numB = 0
    numR = 0
    numQ = 0
    numK = 0
    
# PAWN
    
    for square in board.pieces(1, chess.BLACK):
        idx = numpy.unravel_index(square, (8, 8)) 
        numP += 1
        if ((idx[0]-1) < 0): # out of range at bottom
            continue
           
        elif (idx[0] < 8): # in range of bottom
              
            if (flatPos[idx[0] - 1][idx[1]] == 0): # no piece in front
                matrix[0][idx[0]-1][idx[1]] = 1
                
            if ((idx[1] - 1) >= 0): # if left is in range
                if (flatPos[idx[0]-1][idx[1]-1] == -1): # attack left
                    matrix[0][idx[0]-1][idx[1]-1] = 1
                        
            if ((idx[1] + 1) < 8): #if right is in range
                if (flatPos[idx[0]-1][idx[1]+1] == -1): # attack right
                    matrix[0][idx[0]-1][idx[1]+1] = 1
                    
            if (idx[0] == 6 and flatPos[idx[0]-1][idx[1]] == 0 and flatPos[idx[0]-2][idx[1]] == 0): # special move 2 places forwards
                matrix[0][idx[0]-2][idx[1]] = 1
                    
                
            else:
                continue
            
    #outMatrix = numpy.flip(matrix,1) # corrects the matrix to proper black positions
    #return outMatrix

# KNIGHT
    
    for square in board.pieces(2, chess.BLACK):
        idx = numpy.unravel_index(square, (8, 8)) 
        numN += 1
    
        if (idx[0] < 0 or idx[0] >= 8 or idx[1] < 0 or idx[1] >= 8): # out of range
            test = 2
            continue    
    
        if (idx[0]+2 < 8 and idx[1]+1 < 8):
            if (flatPos[idx[0]+2][idx[1]+1] != 1): # up 2 right 1
                matrix[1][idx[0]+2][idx[1]+1] = 1
                
        if (idx[0]+2 < 8 and idx[1]-1 >=0):
            if (flatPos[idx[0]+2][idx[1]-1] != 1): # up 2 left 1
                matrix[1][idx[0]+2][idx[1]-1] = 1
            
        if (idx[0]-2 >= 0 and idx[1]+1 < 8):
            if (flatPos[idx[0]-2][idx[1]+1] != 1): # down 2 right 1
                matrix[1][idx[0]-2][idx[1]+1] = 1
                
        if (idx[0]-2 >= 0 and idx[1]-1 >= 0): 
            if (flatPos[idx[0]-2][idx[1]-1] != 1): # down 2 left 1
                matrix[1][idx[0]-2][idx[1]-1] = 1
                
        if (idx[0]+1 < 8 and idx[1]+2 < 8):
            if (flatPos[idx[0]+1][idx[1]+2] != 1): # up 1 right 2
                matrix[1][idx[0]+1][idx[1]+2] = 1
                
        if (idx[0]+1 < 8 and idx[1]-2 >= 0):
            if (flatPos[idx[0]+1][idx[1]-2] != 1): # up 1 left 2
                matrix[1][idx[0]+1][idx[1]-2] = 1
                
        if (idx[0]-1 >= 0 and idx[1]+2 < 8): 
            if (flatPos[idx[0]-1][idx[1]+2] != 1): # down 1 right 2
                matrix[1][idx[0]-1][idx[1]+2] = 1
                
        if (idx[0]-1 >= 0 and idx[1]-2 >= 0): 
            if (flatPos[idx[0]-1][idx[1]-2] != 1): #down 1 left 2
                matrix[1][idx[0]-1][idx[1]-2] = 1
                
        else:
            continue
    
    

# BISHOP

    for square in board.pieces(3, chess.BLACK):
        idx = numpy.unravel_index(square, (8, 8)) 
        numB += 1
        
    
        if (idx[0] < 0 or idx[0] >= 8): # out of range
            continue
        if (idx[1] < 0 or idx[1] >= 8): # out of range
            continue
        
        else:
            i = 1 
            while (idx[0] + i < 8 and idx[1] + i < 8): # up right
                if (flatPos[idx[0] + i][idx[1] + i] == 1):
                    break
                
                if (flatPos[idx[0] + i][idx[1] + i] == 0): 
                    matrix[2][idx[0]+i][idx[1]+i] = 1
                
                if (flatPos[idx[0] + i][idx[1] + i] == -1): 
                    matrix[2][idx[0]+i][idx[1]+i] = 1
                    break
                
                
                i += 1
            
            i = 1
            while (idx[0] + i < 8 and idx[1] - i >= 0): # up left
                if (flatPos[idx[0]+ i][idx[1] - i] == 1):
                    break
                
                if (flatPos[idx[0] + i][idx[1] - i] == 0):
                    matrix[2][idx[0]+i][idx[1]-i] = 1
                    
                if (flatPos[idx[0] + i][idx[1] - i] == -1):
                    matrix[2][idx[0]+i][idx[1]-i] = 1
                    break
                

                i += 1
            
            i = 1
            while (idx[0] - i >= 0 and idx[1] - i >= 0): # down left
                if (flatPos[idx[0] - i][idx[1] - i] == 1):
                    break
                    
                if (flatPos[idx[0] - i][idx[1] - i] == 0):
                    matrix[2][idx[0]-i][idx[1]-i] = 1
                    
                if (flatPos[idx[0] - i][idx[1] - i] == -1):
                    matrix[2][idx[0]-i][idx[1]-i] = 1
                    break
                
                i += 1
                
            i = 1
            while (idx[0] - i >= 0 and idx[1] + i < 8): # down right
                if (flatPos[idx[0] - i][idx[1] + i] == 1):
                    break
                    
                if (flatPos[idx[0] - i][idx[1] + i] == 0):
                    matrix[2][idx[0]-i][idx[1]+i] = 1
                    
                if (flatPos[idx[0] - i][idx[1] + i] == -1):
                    matrix[2][idx[0]-i][idx[1]+i] = 1
                    break
                
                i += 1
            
 
    

# ROOK
    
    for square in board.pieces(4, chess.BLACK):
        idx = numpy.unravel_index(square, (8, 8)) 
        numR += 1
        
        if (idx[0] < 0 or idx[0] >= 8): # out of range
            continue
        if (idx[1] < 0 or idx[1] >= 8): # out of range
            continue
    
        else:
            i = 1 
            while (idx[0] + i < 8): # up
                if (flatPos[idx[0] + i][idx[1]] == 1):
                    break
                
                if (flatPos[idx[0] + i][idx[1]] == 0): 
                    matrix[3][idx[0]+i][idx[1]] = 1
                
                if (flatPos[idx[0] + i][idx[1]] == -1): 
                    matrix[3][idx[0]+i][idx[1]] = 1
                    break
                
                i += 1
                
            i = 1 
            while (idx[0] - i >= 0): # down
                if (flatPos[idx[0] - i][idx[1]] == 1):
                    break
                
                if (flatPos[idx[0] - i][idx[1]] == 0): 
                    matrix[3][idx[0]-i][idx[1]] = 1
                
                if (flatPos[idx[0] - i][idx[1]] == -1): 
                    matrix[3][idx[0]-i][idx[1]] = 1
                    break
                
                i += 1
                
            i = 1 
            while (idx[1] + i < 8): # right
                if (flatPos[idx[0]][idx[1]+i] == 1):
                    break
                
                if (flatPos[idx[0]][idx[1]+i] == 0): 
                    matrix[3][idx[0]][idx[1]+i] = 1
                
                if (flatPos[idx[0]][idx[1]+i] == -1): 
                    matrix[3][idx[0]][idx[1]+i] = 1
                    break
                
                i += 1
                
            i = 1 
            while (idx[1] - i >= 0): # right
                if (flatPos[idx[0]][idx[1]-i] == 1):
                    break
                
                if (flatPos[idx[0]][idx[1]-i] == 0): 
                    matrix[3][idx[0]][idx[1]-i] = 1
                
                if (flatPos[idx[0]][idx[1]-i] == -1): 
                    matrix[3][idx[0]][idx[1]-i] = 1
                    break
                
                i += 1
    
    


# QUEEN

    for square in board.pieces(5, chess.BLACK):
        idx = numpy.unravel_index(square, (8, 8)) 
        numQ += 1

        if (idx[0] < 0 or idx[0] >= 8): # out of range
            continue
        if (idx[1] < 0 or idx[1] >= 8): # out of range
            continue
        
        else:
            i = 1 
            while (idx[0] + i < 8 and idx[1] + i < 8): # up right
                if (flatPos[idx[0] + i][idx[1] + i] == 1):
                    break
                
                if (flatPos[idx[0] + i][idx[1] + i] == 0): 
                    matrix[4][idx[0]+i][idx[1]+i] = 1
                
                if (flatPos[idx[0] + i][idx[1] + i] == -1): 
                    matrix[4][idx[0]+i][idx[1]+i] = 1
                    break
                
                
                i += 1
            
            i = 1
            while (idx[0] + i < 8 and idx[1] - i >= 0): # up left
                if (flatPos[idx[0]+ i][idx[1] - i] == 1):
                    break
                
                if (flatPos[idx[0] + i][idx[1] - i] == 0):
                    matrix[4][idx[0]+i][idx[1]-i] = 1
                    
                if (flatPos[idx[0] + i][idx[1] - i] == -1):
                    matrix[4][idx[0]+i][idx[1]-i] = 1
                    break
                

                i += 1
            
            i = 1
            while (idx[0] - i >= 0 and idx[1] - i >= 0): # down left
                if (flatPos[idx[0] - i][idx[1] - i] == 1):
                    break
                    
                if (flatPos[idx[0] - i][idx[1] - i] == 0):
                    matrix[4][idx[0]-i][idx[1]-i] = 1
                    
                if (flatPos[idx[0] - i][idx[1] - i] == -1):
                    matrix[4][idx[0]-i][idx[1]-i] = 1
                    break
                
                i += 1
                
            i = 1
            while (idx[0] - i >= 0 and idx[1] + i < 8): # down right
                if (flatPos[idx[0] - i][idx[1] + i] == 1):
                    break
                    
                if (flatPos[idx[0] - i][idx[1] + i] == 0):
                    matrix[4][idx[0]-i][idx[1]+i] = 1
                    
                if (flatPos[idx[0] - i][idx[1] + i] == -1):
                    matrix[4][idx[0]-i][idx[1]+i] = 1
                    break
                
                i += 1
            

            i = 1 
            while (idx[0] + i < 8): # up
                if (flatPos[idx[0] + i][idx[1]] == 1):
                    break
                
                if (flatPos[idx[0] + i][idx[1]] == 0): 
                    matrix[4][idx[0]+i][idx[1]] = 1
                
                if (flatPos[idx[0] + i][idx[1]] == -1): 
                    matrix[4][idx[0]+i][idx[1]] = 1
                    break
                
                i += 1
                
            i = 1 
            while (idx[0] - i >= 0): # down
                if (flatPos[idx[0] - i][idx[1]] == 1):
                    break
                
                if (flatPos[idx[0] - i][idx[1]] == 0): 
                    matrix[4][idx[0]-i][idx[1]] = 1
                
                if (flatPos[idx[0] - i][idx[1]] == -1): 
                    matrix[4][idx[0]-i][idx[1]] = 1
                    break
                
                i += 1
                
            i = 1 
            while (idx[1] + i < 8): # right
                if (flatPos[idx[0]][idx[1]+i] == 1):
                    break
                
                if (flatPos[idx[0]][idx[1]+i] == 0): 
                    matrix[4][idx[0]][idx[1]+i] = 1
                
                if (flatPos[idx[0]][idx[1]+i] == -1): 
                    matrix[4][idx[0]][idx[1]+i] = 1
                    break
                
                i += 1
                
            i = 1 
            while (idx[1] - i >= 0): # right
                if (flatPos[idx[0]][idx[1]-i] == 1):
                    break
                
                if (flatPos[idx[0]][idx[1]-i] == 0): 
                    matrix[4][idx[0]][idx[1]-i] = 1
                
                if (flatPos[idx[0]][idx[1]-i] == -1): 
                    matrix[4][idx[0]][idx[1]-i] = 1
                    break
                
                i += 1
        
        
    
        
# KING
    
    for square in board.pieces(6, chess.BLACK):
        idx = numpy.unravel_index(square, (8, 8)) 
        numK += 1

        if (idx[0] < 0 or idx[0] >= 8): # out of range
            continue
        if (idx[1] < 0 or idx[1] >= 8): # out of range
            continue
        
        
        if (idx[0] + 1 < 7):  
                if (flatPos[idx[0]+1][idx[1]] != 1): # up
                    matrix[5][idx[0]+1][idx[1]] = 1
        
        if (idx[1] + 1 < 7 and idx[0] + 1 < 7):
                if (flatPos[idx[0]+1][idx[1]+1] != 1): # up right
                    matrix[5][idx[0]+1][idx[1]+1] = 1
                    
        if (idx[1] + 1 < 7):
                if (flatPos[idx[0]][idx[1]+1] != 1): # right
                    matrix[5][idx[0]][idx[1]+1] = 1
                    
        if (idx[0] - 1 >= 0 and idx[1] + 1 < 7):
                if (flatPos[idx[0]-1][idx[1]+1] != 1): # down right
                    matrix[5][idx[0]-1][idx[1]+1] = 1
                    
        if (idx[0] - 1 >= 0):
                if (flatPos[idx[0]-1][idx[1]] != 1): # down 
                    matrix[5][idx[0]-1][idx[1]] = 1
                    
        if (idx[0] - 1 >= 0 and idx[1] - 1 >= 0):
                if (flatPos[idx[0]-1][idx[1]-1] != 1): # down left
                    matrix[5][idx[0]-1][idx[1]-1] = 1
                    
        if (idx[1] - 1 >= 0):
                if (flatPos[idx[0]][idx[1]-1] != 1): # left
                    matrix[5][idx[0]][idx[1]-1] = 1
                    
        if (idx[0] + 1 < 7 and idx[1] - 1 >= 0):
                if (flatPos[idx[0]+1][idx[1]-1] != 1): # up left
                    matrix[5][idx[0]+1][idx[1]-1] = 1
    
    numBPieces[0] = numP
    numBPieces[1] = numN
    numBPieces[2] = numB
    numBPieces[3] = numR
    numBPieces[4] = numQ
    numBPieces[5] = numK
    
    outMatrix = numpy.flip(matrix,1) # corrects the matrix to proper black positions
    flatPos = numpy.flip(flatPos,0)
    return outMatrix, flatPos, numBPieces

In [ ]:
# 
def evalFunc(numWPieces, newW, row, col):
    val = 0
    mobility = 0
    #Bcheck = 0
    
    if (row > 1 and row < 6):
        if(col > 1 and col < 6): # center squares
            mobility = 12
        if (col == 1 or col == 6):
            mobility = 10
            if (row == col):
                mobility = mobility - 2
        if (col == 0 or col == 7):
            mobility = 6
            if (row == col):
                mobility = mobility - 2
                
    if (col > 1 and col < 6):
        if (row == 1 or row == 6):
            mobility = 10
            if (row == col):
                mobility = mobility - 2
        if (row == 0 or row == 7):
            mobility = 6
            if (row == col):
                mobility = mobility - 2
                
                
    if (col < 2 or col > 5):
        if (row < 2 or row > 5):
            mobility = 6
        if (row == 0):
            if (col == 0 or col == 7):
                mobility = mobility - 2
        if (row == 7):
            if (col == 0 or col == 7):
                mobility = mobility - 2
                
    #ADD WHITE ATTACKS LATER
    
    #board.turn = chess.BLACK
    #current_turn = board.turn
    #if(board.is_check()):
        #Bcheck = -200
                
            
    
    
    val = 200*(numWPieces[5] - newW[5])+(
            9*(numWPieces[4] - newW[4])+
            5*(numWPieces[3] - newW[3])+
            3*(numWPieces[2] - newW[2])+
            3*(numWPieces[1] - newW[1])+
            1*(numWPieces[0] - newW[0])+
            mobility)
    
    #val = 200(netK)
           #+ 9(netQ)
           #+ 5(netR)
           #+ 3(netB + netN)
           #+ 1(netP)
    
    return val

In [ ]:
# MOVE EVALUATION FUNCTION

def evalMove(board):
    #board is of size (6,8,8)
    eval2d = numpy.zeros((6, 8, 8), dtype=numpy.float32) # 6 by 8 by 8 vector of all 0's
    Normeval2d = numpy.zeros((6, 8, 8), dtype=numpy.float32) # 6 by 8 by 8 vector of all 0's

    
    moves = splitLegalMove(board)[0]
    flatPos = splitLegalMove(board)[1]
    numBPieces = splitLegalMove(board)[2]

    numWPieces = numpy.zeros(6, dtype=numpy.int8)
    tempW = numpy.zeros(6, dtype=numpy.int8)
    tempB = numpy.zeros(6, dtype=numpy.int8)

    
    Ptype = 0
    
    for i in range (6):
        for square in board.pieces(i+1, chess.WHITE): # get number of white pieces by type
            numWPieces[i] += 1
        tempW[i] = numWPieces[i]
        #t = numWPieces[i]
        #tempW[i] = t
            

            
    # Actual evaluation of each legal move square
    for i in range (6): # piece
        for j in range (8): # row
            for k in range (8): # col
                numWPieces = numpy.zeros(6, dtype=numpy.int8)
                for a in range (6):
                    for square in board.pieces(a+1, chess.WHITE): # get number of white pieces by type
                        numWPieces[a] += 1
                    tempW[a] = numWPieces[a]
                
                if (moves[i][j][k] == 1): # potential move square #moves
                    if (flatPos[j][k] == -1):
                        Ptype = board.piece_type_at(chess.square(k, 7-j)) # chess.square is col major (c,r) and reversed row
                        if (Ptype == 'None'):
                            continue
                        if (Ptype == 1): # Pawn
                            tempW[0] = numWPieces[0] - 1
                        if (Ptype == 2): # Knight
                            tempW[1] = numWPieces[1] - 1
                        if (Ptype == 3): # Bishop
                            tempW[2] = numWPieces[2] - 1
                        if (Ptype == 4): # Rook
                            tempW[3] = numWPieces[3] - 1
                        if (Ptype == 5): # Queen
                            tempW[4] = numWPieces[4] - 1
                        if (Ptype == 6): # King
                            tempW[5] = numWPieces[5] - 1
                        
                    eval2d[i][j][k] = evalFunc(numWPieces, tempW, j, k)
                    
# NORMALIZATION 
        # sum of all squares will equal 1
        # will not work for negative numbers

        for i in range (6): # piece
            for j in range (8): # row
                for k in range (8): # col
                    Normeval2d[i][j][k] = eval2d[i][j][k] / numpy.sum(eval2d)
                            
    
    return  Normeval2d

In [ ]:
# CONVERTS FILE OF GAMES INTO AN ARRAY OF GAMES
pgn = open("C:\\Users\\travi\\Downloads\\Chess_data\\Chess_RAW_blitz_games.txt")

gameList = [] # initialize array

while True:
    game = chess.pgn.read_game(pgn) # read the file
    if game is None: # checks if at end of file
        break  # end of file, stop reading

    gameList.append(game) # adds each game to a location in an array

In [ ]:
# NPZ FILE OF SPLIT GAMES & DETERMINED NORMALIZED VALUES
#size = len(gameList)

def data2NPZ():
    games = 25000
    numMoves = 0



    for k in range (games):
        board = gameList[k].board()
        for move in gameList[k].mainline_moves(): # get expected size of array based on number of games
            board.push(move)
        numMoves += board.fullmove_number
    size = numMoves


    i = 0
    k = 0
    outMatrix = numpy.zeros((size, 6, 8, 8), dtype=numpy.int8)
    Yarray = numpy.zeros((size, 6, 8, 8), dtype=numpy.float32)


    with open('C:\\Users\\travi\\Downloads\\Chess_data\\XYTrainingData25K.npz', 'wb') as f:
        while i < (games):
            turn_cnt = 0 # start turn as white
            board = gameList[i].board() # resets the board each game
            for move in gameList[i].mainline_moves(): # iterates through all moves in the game
                board.push(move) # inputs move from game on board
                if (turn_cnt % 2 == 1): # black's turn
                    splitMatrix = split_dims(board) # converts boardstate to matrices
                    outMatrix[k] = splitMatrix # adds individual move matrices to a collection
                    evalArray = evalMove(board)
                    Yarray[k] = evalArray
                    k += 1
                turn_cnt += 1
            
            i += 1
        numpy.savez(f, x = outMatrix, y = Yarray)
    f.close()
    
    return None

In [ ]:
data2NPZ()